In [ ]:
import pandas as pd
import re


# course_data = pd.read_excel('course_data.xlsx')
# course_data = course_data[['Course Title', 'Tags']]
# course_data = course_data.drop_duplicates()
# course_data = course_data.dropna()
# course_data = course_data[~course_data.apply(lambda x: x.astype(str).str.contains(r'^\[\]')).any(axis=1)]
# course_data['Tags'] = course_data['Tags'].apply(lambda x: re.sub(r'[^a-zA-Z\sа-яА-ЯёЁ]', '', x))
# course_data['Tags'] = course_data['Tags'].str.lower()
# course_data

,Course Title,Tags
0,Motion design,координация цвета стили материалы ткани и акс...
1,UI/UX design,дизайн и макетирование webстраниц подготовка г...
2,Копирайтинг,маркетинговая деятельность проведение рекламны...
3,Кәсіби визаж,нанесение макияжа подготовка кожи для нанесени...
4,Көкөністерден әзірленген тағамдар,приготовление полуфабрикатов из рыбы изготовле...
...,...,...
590,Baipket Academy Bastapket,управление деятельностью стратегическое мышлен...
597,Технология «БиС» для учителей,управление деятельностью принятие решений упра...
599,Курс Арбитраж ставок финансирования,стратегическое мышление
600,LUX QURAQ подушка (3 поток),инициативность ответственность ориентация на п...


In [14]:
import pandas as pd
import re


course_data = pd.read_csv('course_data3.csv')
course_data = course_data[['Course Title', 'Unnamed: 1']]
course_data = course_data.drop_duplicates()
course_data = course_data.dropna()
# course_data = course_data[~course_data.apply(lambda x: x.astype(str).str.contains(r'^\[\]')).any(axis=1)]
# course_data['Tags'] = course_data['Tags'].apply(lambda x: re.sub(r'[^a-zA-Z\sа-яА-ЯёЁ]', '', x))
course_data['Unnamed: 1'] = course_data['Unnamed: 1'].str.lower()
course_data = course_data.rename(columns = {'Unnamed: 1': 'Tags'})

In [15]:
course_data

,Course Title,Tags
0,Motion design,моушн дизайн анимация графика видео движение м...
1,UI/UX design,интерфейс дизайн пользовательский опыт проекти...
2,Копирайтинг,копирайтинг текст написание статьи контент seo...
3,Кәсіби визаж,визаж макияж косметика мейкап бьюти красота пр...
4,Көкөністерден әзірленген тағамдар,овощи кулинария приготовление вегетарианский в...
...,...,...
553,Мұғалімдерге арналған «Бис» технологиясы,образование методика бис учителя педагогика те...
554,Детский курс “Құрақ көрпе” стандарт,курак корпе рукоделие детский курс творчество ...
555,Детский курс “Құрақ көрпе” VIP,курак корпе рукоделие детский курс творчество vip
557,Директорларға арналған «БиС» технологиясы,образование управление методика бис директора ...


In [16]:
from transformers import AutoTokenizer, AutoModel
import torch

MODEL_NAME = "cointegrated/rubert-tiny"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

## Embedding Generation

def get_embedding(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [17]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_courses(input_embedding, tag_embeddings, course_titles, top_n=3):

    # input_embedding = get_embedding(input)

    # Ensure input_embedding is 2D for sklearn's cosine_similarity
    if input_embedding.ndim == 1:
        input_embedding = input_embedding.reshape(1, -1)

    # Calculate cosine similarity between input and all tag embeddings
    similarities = cosine_similarity(input_embedding, tag_embeddings).flatten()

    # Get indices of top N similar items
    top_indices = np.argsort(similarities)[::-1][:top_n]

    # Create list of (course_title, similarity_score) tuples
    results = [(course_titles[i], float(similarities[i])) for i in top_indices]

    return results

In [18]:
# Preparing embedding of tas and list of course titles
tag_embeddings = np.array([get_embedding(tags) for tags in course_data['Tags']])
course_titles = course_data['Course Title'].tolist()

In [22]:
def clean_text(text):
    # Import string to get punctuation
    import string

    # Convert to lowercase and remove punctuation
    text = text.lower()
    # Remove all punctuation marks
    text = ''.join(char for char in text if char not in string.punctuation)

    return text

In [42]:
# Get embedding for input query
query = "бухгалтер"
query = clean_text(query)
query_embedding = get_embedding(query)


# Find similar courses
similar_courses = find_similar_courses(
    query_embedding,
    tag_embeddings,
    course_titles
)

# Print results
for title, score in similar_courses:
    print(f"Course: {title}")



Course: Сертифицированный главный бухгалтер (МСФО)
Similarity: 0.7329

Course: Бастаушы Бухгалтер
Similarity: 0.6995

Course: Возврат НДС-основные моменты и ситуации
Similarity: 0.6945

